In [67]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import math
import seaborn as sns

In [68]:
import sys
print(sys.version)

3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


Imported system version

In [69]:
import pandas as pd
data = pd.read_csv("convert_churn_ds.csv")
data

,CustomerID,Tenure,PhoneService1,Contract1,PaymentMethod1,MonthlyCharges,TotalCharges,Churn1,TotalchargesPercontract,log_monthly_charges
0,7590-VHVEG,1,0,0,0,29.85,29.85,0,inf,3.396185
1,5575-GNVDE,34,1,1,1,56.95,1936.30,0,1936.30,4.042174
2,3668-QPYBK,2,1,0,1,53.85,107.70,1,inf,3.986202
3,7795-CFOCW,45,0,1,2,42.30,1903.50,0,1903.50,3.744787
4,9237-HQITU,2,1,0,0,70.70,141.40,1,inf,4.258446
...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,24,1,1,1,84.80,2035.20,0,2035.20,4.440296
7039,2234-XADUH,72,1,1,3,103.20,7430.40,0,7430.40,4.636669
7040,4801-JZAZL,11,0,0,0,29.60,325.60,0,inf,3.387774
7041,8361-LTMKD,4,1,0,1,74.40,297.60,1,inf,4.309456


In [70]:
import numpy as np
import pandas as pd


data.replace([np.inf, -np.inf], 0, inplace=True)

# Now you can proceed with your PyCaret setup
from pycaret.classification import *
automl = setup(data, target='Churn1')

,Description,Value
0,Session id,526
1,Target,Churn1
2,Target type,Binary
3,Original data shape,"(7043, 10)"
4,Transformed data shape,"(7043, 10)"
5,Transformed train set shape,"(4930, 10)"
6,Transformed test set shape,"(2113, 10)"
7,Numeric features,8
8,Categorical features,1
9,Preprocess,True


In [71]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.7667,0.7693,0.4557,0.5787,0.5079,0.3584,0.3638,0.0670
svm,SVM - Linear Kernel,0.7659,0.7599,0.4344,0.6261,0.4618,0.3320,0.3651,0.0540
lr,Logistic Regression,0.7430,0.8188,0.0542,0.7249,0.0997,0.0649,0.1435,0.2110
nb,Naive Bayes,0.7422,0.8077,0.6987,0.5111,0.5897,0.4085,0.4197,0.0380
dt,Decision Tree Classifier,0.7347,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0380
ridge,Ridge Classifier,0.7347,0.8206,0.0000,0.0000,0.0000,0.0000,0.0000,0.0510
rf,Random Forest Classifier,0.7347,0.5940,0.0000,0.0000,0.0000,0.0000,0.0000,0.2010
qda,Quadratic Discriminant Analysis,0.7347,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0440
ada,Ada Boost Classifier,0.7347,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0450
gbc,Gradient Boosting Classifier,0.7347,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2030


In [72]:
best_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [103]:
data.iloc[-2:-1]

,CustomerID,Tenure,PhoneService_Numeric,Contract_Numeric,PaymentMethod_Numeric,MonthlyCharges,TotalCharges,Churn_Numeric,MonthlyChargesPerTenure,LogTotalCharges
3,4,28,1,0,0,104.8,3046.05,1,3.742857,8.021601


In [105]:
data.iloc[-2:-1].shape

(1, 10)

In [107]:
data.iloc[-1].shape

(10,)

In [76]:
predict_model(best_model, data.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,CustomerID,Tenure,PhoneService1,Contract1,PaymentMethod1,MonthlyCharges,TotalCharges,TotalchargesPercontract,log_monthly_charges,Churn1,prediction_label,prediction_score
7041,8361-LTMKD,4,1,0,1,74.400002,297.600006,0.0,4.309456,1,1,0.8


In [109]:
save_model(best_model, 'bestmodel')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Tenure', 'PhoneService1',
                                              'Contract1', 'PaymentMethod1',
                                              'MonthlyCharges', 'TotalCharges',
                                              'TotalchargesPercontract',
                                              'log_monthly_charges'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_va...
                                     transformer=TargetEncoder(cols=['CustomerID'],
        

In [111]:
import pickle
with open('bestmodel.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [113]:
with open('bestmodel.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [115]:
loaded_lda = load_model('bestmodel')

Transformation Pipeline and Model Successfully Loaded


In [117]:
new_data=data.iloc[-2:-1]

In [121]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model
from sklearn.preprocessing import LabelEncoder

def load_data(filepath):
    """
    Loads churn data into a DataFrame from a string filepath.
    """
    data = pd.read_csv(filepath)

    # Preprocess 'CustomerID' column if needed
    le = LabelEncoder()
    data['CustomerID_Encoded'] = le.fit_transform(data['CustomerID'])
    data['CustomerID'] = data['CustomerID_Encoded']
    data.drop(columns=['CustomerID_Encoded'], inplace=True)

    return data

def make_predictions(data, threshold=0.5):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    Rounds up to 1 if greater than or equal to the threshold.
    """
    predictions = predict_model(model, data=data)
    predictions['Churn_prediction'] = (predictions['Churn_Numeric'] >= threshold)
    predictions['Churn_prediction'].replace({True: 'Churn', False: 'No churn'}, inplace=True)
    drop_cols = predictions.columns.tolist()
    drop_cols.remove('Churn_prediction')
    return predictions.drop(drop_cols, axis=1)

if __name__ == "__main__":
    model = load_model('LinDiscrAnalysis')  # Assuming the model is named 'LinDiscrAnalysis'
    data = load_data('new_churn_data.csv')  # Replace with your data path
    predictions = make_predictions(data)
    print('predictions:')
    print(predictions)

In [123]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


predictions:
  Churn_prediction
0            Churn
1         No churn
2         No churn
3            Churn
4         No churn


# Summary:

I imported all necessary packages then read my csv file from week 2 and used pycarat classification to set my target as churn and  then found that the best model is  KNeighborsClassifier it gave a 100% accuracy score and saved my model for further predictions. i then used predict_churn python code file for making predictions on the new_churndata that i have taken as a subset from my original data which gave an accurate result.